In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install segmentation_models

In [ ]:
!pip install -U -q segmentation-models

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

from segmentation_models import Unet

Segmentation Models: using `tf.keras` framework.


In [ ]:

image_folder_path = ('/content/drive/MyDrive/Bootcamp_competition_2023/images/train_images')
csv_file_path = ('/content/drive/MyDrive/Bootcamp_competition_2023/train.csv')
data = pd.read_csv(csv_file_path)



In [ ]:
"""
!pip install rembg
from rembg import remove

filename = '8e088243586ca5318ab22e73d4059c52.jpg'
input_folder = image_folder_path

if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
      input_path = os.path.join(input_folder, filename)
      output_path = os.path.join(output_folder, filename)

        # Open the input image and remove the background
      with open(input_path, 'rb') as inp_file, open(output_path, 'wb') as out_file:
          img = inp_file.read()
          out_img = remove(img)
          out_file.write(out_img)

      print(f"Processed: {filename}")

print("Background removal completed.")
"""

In [ ]:
"""
import os
from PIL import Image
folder_path = output_folder
extensions = []
for filee in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filee)
      print('** Path: {}  **'.format(file_path), end="\r", flush=True)
      im = Image.open(file_path)
      rgb_im = im.convert('RGB')
      if filee.split('.')[1] not in extensions:
          extensions.append(filee.split('.')[1])
"""

In [ ]:
!pip install tensorflow-addons
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization, Add, Input
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.applications import ResNet50
from tensorflow_addons.metrics import F1Score
import os
from keras.applications.efficientnet import EfficientNetB6

In [ ]:
train_data, validation_data = train_test_split(data, test_size=0.2, random_state=42)
image_size = (224, 224)

train_data['class_6'] = train_data['class_6'].astype(str)
validation_data['class_6'] = validation_data['class_6'].astype(str)
"""
# Генератори даних
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    #rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    #shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest'
)
"""
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    #rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=image_folder_path,
    x_col='image_id',
    y_col='class_6',
    target_size=image_size,
    batch_size=16,
    class_mode='categorical'
)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_generator = valid_datagen.flow_from_dataframe(
    dataframe=validation_data,
    directory=image_folder_path,
    x_col='image_id',
    y_col='class_6',
    target_size=image_size,
    batch_size=16,
    class_mode='categorical'
)

In [ ]:
import matplotlib.pyplot as plt
def plot_images(image_generator, num_images=3):
    batch, labels = next(image_generator)

    indices = range(num_images)
    selected_images = [batch[i] for i in indices]
    selected_labels = [labels[i] for i in indices]

    plt.figure(figsize=(10, 6))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(selected_images[i])
        plt.title(f'Class: {selected_labels[i].argmax()}')
        plt.axis('off')
    plt.show()

plot_images(train_generator)

plot_images(validation_generator)


In [ ]:

base_model = EfficientNetB6(input_shape = (224,224,3), weights='imagenet', include_top=False)
for layer in base_model.layers[-10:]:
    layer.trainable = True
x = base_model.output


x = Conv2D(512, (3, 3), activation='relu', padding = 'valid')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.4)(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Dropout(0.4)(x)

x = Dense(512, activation = 'relu',  kernel_regularizer=l2(0.007))(x)
x = Dropout(0.5)(x)
x = Dense(128, activation = 'relu',  kernel_regularizer=l2(0.007))(x)
x = GlobalAveragePooling2D()(x)
predictions = Dense(6, activation = 'softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions)


optimizer = Adam(learning_rate=0.001)

scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)


model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = [F1Score(num_classes=6, average="micro")])


history = model.fit(train_generator, epochs = 32, validation_data = validation_generator, callbacks=[scheduler, early_stopping])
model.save_weights('final_model_weights.h5')

In [ ]:
model.load_weights('final_model_weights.h5')
history = model.fit(train_generator, epochs = 40, validation_data = validation_generator)

In [ ]:
def accuracy_loss_plot(history):
  plt.figure(figsize = (12,4))

  plt.subplot(1,2,1)
  plt.plot(history.history['f1_score'], label = 'Training F1')
  plt.plot(history.history['val_f1_score'], label = 'Validation F1')
  plt.title('Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')

  plt.subplot(1,2,2)
  plt.plot(history.history['loss'], label = 'Training loss')
  plt.plot(history.history['val_loss'], label = 'Validation loss')
  plt.title('Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')

  plt.tight_layout()
  plt.show()

accuracy_loss_plot(history)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

num_of_test_samples = 472
batch_size = 64

Y_pred = model.predict(validation_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['0', '1', '2', '3', '4', '5']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))